In [225]:
import string
import pandas as pd
import nltk
import eng_to_ipa as ipa
import spacy
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [226]:
NER = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [227]:
countable_nouns = pd.read_csv('countable_nouns.csv')
uncountable_nouns = pd.read_csv('uncountable_nouns.csv')
countable = countable_nouns['countable_nouns'].tolist()
uncountable = uncountable_nouns['uncountable_nouns'].tolist()

In [228]:
#Function for detecting vowel sound using phonetics
def vowel_sound_check(string):
    vowels = ['ə', 'i', 'aɪ', 'oʊ', 'ɔ', 'ɪ','æ','ɛ','ʊ','u','ɑ','aɪ','aʊ','a']
    str_ipa = ipa.convert(string)
    if str_ipa.startswith("ˈ") or str_ipa.startswith("ˌ"):
        if str_ipa[1] in vowels or str_ipa[1:3] in vowels:
            return 1
    elif str_ipa[0] in vowels or str_ipa[0:2] in vowels:
            return 1
    else:
        return 0

In [340]:
Adjective_exceptions = ['Whole','whole','first','First','Same','same','few','Few','Little','little','all','All','Both','both','Heavy','heavy','Such','such','Big','big']

In [362]:
def Article_Error_Correction(text):
    
    output = ''
    
    raw_words= word_tokenize(text)
    iob = pos_tag(raw_words)
    
    #print('iob1->',iob)
    
    
    #Aritcle Omission Correction
    seen = list()
    for i in range(len(iob)):
        seen.append(iob[i][0])
        
    to_re_tag=''
    
    for i in range(len(iob)):
    #Removing Extra Article from sentence
        if i+1<len(iob) and iob[i][0] in ['The','the','A','a','An','an'] and iob[i+1][1] not in ['NN','NNP','JJ','JJR','JJS','RB','RBR']:
            seen[i] = ''
    for i in range(len(seen)):
        if i+1<len(seen) and (seen[i+1] not in ['.',',','?','!',';',"'",':','(',')','[',']','/'] and seen[i+1].startswith("'")==False) and seen[i]!='':
            to_re_tag+=seen[i]+' '
        else:
            to_re_tag+=seen[i]
    
    #POS tagging after removal of extra article
    raw_words= word_tokenize(to_re_tag)
    tags = pos_tag(raw_words)
    
    #Chunking
    res_chunk = ne_chunk(tags)
    iob = tree2conlltags(res_chunk)
    iob = list([x[:len(x)-1] for x in iob])
    
    #Applying NER (NLTK and Spacy)
    NER_text= NER(to_re_tag)
    NER_tags = list()
    for item in iob:
        item=list(item)
        for w in NER_text.ents:
            if w.text in item:
                item.append(w.label_)
        item=tuple(item)
        NER_tags.append(item)
    iob = NER_tags
    
    #print('iob2->',iob)
    
    
    seen = list()
    for i in range(len(iob)):
        seen.append(iob[i][0])
        
    
    
    
    #Main Loop
    for i in range(len(iob)):
        
        
        
        #1. FOR NOUNS
        #1.1 if previous pos tag is IN - no article used before noun
        if iob[i][1]=='NNP' and iob[i-1][1] not in ['JJR','JJS','RBR'] and iob[i-1][0] not in ['The','the']:
            if i+1<len(iob):
                if iob[i+1][1] not in ['POS','VBD','VBZ'] and iob[i-1][1] not in ['NNP','IN'] :
                    seen[i] = 'the '+iob[i][0]
                    if iob[i-1][0] in ['A','a','An','an']: 
                        eliminate_prev(i,seen)
                        
                elif iob[i-1][1] == 'IN' and iob[i+1][1] in ['NN','NNPS']:
                    seen[i] = 'the '+iob[i][0]
                    if iob[i-1][0] in ['A','a','An','an']: 
                        eliminate_prev(i,seen)
                        
            elif i+2<len(iob) and iob[i+1] in ['IN'] and iob[i+2] in ['NNP']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                        
        #1.3 NER - no ->
        #1.3.1 singular & countable noun ->
        elif iob[i][1] =='NN' and iob[i][0] in countable:
            if vowel_sound_check(iob[i][0]) and iob[i-1][0] not in ['An','an']:
                seen[i] = 'an '+iob[i][0]
                if iob[i-1][0] in ['A','a','The','the']: 
                    eliminate_prev(i,seen)
                    
            #POS of previous token is IN (of,in) or noun is used after WP or WRB(What's,Where's) -> the
            elif iob[i-1][1] in ['IN'] or (iob[i-1][1]=='POS' and iob[i-2][1] in ['WP','WRB']) and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif i+2<len(iob) and iob[i+1][1] in ['IN'] and iob[i+2][1] in ['NNP'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif i+1<len(iob) and iob[i-1][1] in ['VBP','VBG'] and iob[i+1][1] in ['IN'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif i+1<len(iob) and iob[i+1][1] in ['PRP'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            #When the POS of previous token is not in following list -> a
            elif iob[i-1][1] not in ['DT','JJ','JJR','JJS','PRP','PRP$','NN','NNP'] and iob[i-2][1] not in ['RB','RBR'] and iob[i-1][0] not in ['A','a']:
                seen[i] = 'a '+iob[i][0]
                if iob[i-1][0] in ['The','the','An','an']: 
                    eliminate_prev(i,seen)
            else:
                #print('(no article is ok(singular and countable)) ->'+iob[i][0] )
                pass
                
        #1.3.2 plural or uncountable
        elif iob[i][1]=='NNS' or iob[i][0].lower() in uncountable:
            if iob[i][0] in ['east','west','north','south','north-east','north-west','south-west','south-east'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif iob[i-1][1] == 'IN' and iob[i-2][1] in ['JJS'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif iob[i][0] in countable and iob[i][1] not in ['VBG','VBN','VBZ','VBD'] and iob[i-1][0] not in ['A','a']:
                seen[i] = 'a '+iob[i][0]
                if iob[i-1][0] in ['The','the','An','an']: 
                    eliminate_prev(i,seen)
            else:
                #print('(no article is ok (plural or uncountable)) ->'+iob[i][0])
                pass
        else:
            pass





        #ORDINAL NUMBERS (RB,JJ,NN)
        flag=0
        if 2<len(iob[i]):
            if i+1<len(iob) and iob[i][2]=='ORDINAL' and iob[i-1][1] not in ['PRP'] and iob[i+1][1] not in ['.'] and iob[i-1][0] not in ['the','The']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                flag=1
            elif iob[i-1][0] == 'the':
                eliminate_prev(i,seen)
            if flag==1:
                break



        #2.FOR ADJECTIVES
        #2.1 Before superlative degree of adjective -> the
        if iob[i][1]=='JJS' and iob[i-1][1] not in ['DT']:
            seen[i] = "the "+iob[i][0]
            if iob[i-1][0] in ['A','a','An','an']: 
                eliminate_prev(i,seen)

        #2.2 Before comparative degree of adjective - next pos token is IN(Eg. of) -> the
        elif iob[i][1] == 'JJR':
            if i+1<len(iob) and iob[i+1][1] in ['PRP'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif i+1<len(iob) and iob[i+1][0] in ['of'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif iob[i-1][1] == 'VBZ':
                if i+1<len(iob) and iob[i+1][1] == 'NNP':
                    seen[i] = "a "+iob[i][0]
                    if iob[i-1][0] in ['The','the','An','an']: 
                        eliminate_prev(i,seen)
            else:
                #print("no article is ok(comparitive)")
                pass

        

        #2.3. Different format for use of article with adjective( 'DT JJ Noun','DT JJ Verb','NNP DT JJ', Exceptions)
        elif iob[i][1]=='JJ':
            if vowel_sound_check(iob[i][0]) and iob[i-1][0] not in ['An','an']: 
                    seen[i] = "an "+iob[i][0]
                    if iob[i-1][0] in ['A','a','The','the']: 
                        eliminate_prev(i,seen)
                        
            elif i+1<len(iob) and iob[i+1][1] in ['NNP'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)
                    
            elif i+1<len(iob) and iob[i+1][1] in ['NN','VB']:
                if i+1<len(iob) and iob[i+1][1]=='VB' and iob[i-1][0] not in ['The','the']:
                    seen[i] = 'the '+iob[i][0]
                    if iob[i-1][0] in ['A','a','An','an']: 
                        eliminate_prev(i,seen)
                elif iob[i-2][1] in ['RB'] and iob[i-1][0] in 'a':
                    eliminate_prev(i,seen)
                        
                elif iob[i-1][1] not in ['RB'] and iob[i-1][0] not in ['A','a'] and seen[i-1].endswith('a')==False and iob[i][0] not in Adjective_exceptions:
                    seen[i] = 'a '+iob[i][0]
                    if iob[i-1][0] in ['The','the','An','an']: 
                        eliminate_prev(i,seen)
                    
            elif iob[i-1][1] in ['NNP'] and iob[i-1][0] not in ['The','the']:
                    seen[i] = "the "+iob[i][0]
                    if iob[i-1][0] in ['A','a','An','an']: 
                        eliminate_prev(i,seen)
            
            elif iob[i][0] in ['whole','same'] and iob[i-1][0] not in ['The','the']:
                seen[i] = "the "+iob[i][0]
                if iob[i-1][0] in ['A','a','An','an']: 
                    eliminate_prev(i,seen)

            elif iob[i][0] in ['few','little'] and iob[i-1][0] not in ['A','a']:
                seen[i] = "a "+iob[i][0]
                if iob[i-1][0] in ['An','an','The','the']: 
                    eliminate_prev(i,seen)

            elif iob[i][0] in ['all','both'] and iob[i-1][0] not in ['The','the']:
                print(1)
                seen[i] = iob[i][0]+" the"
                #eliminate next case 
                if i+1<len(iob) and iob[i+1][0] in ['A','a','An','an']: 
                    eliminate_next(i)

            elif i+1<len(iob) and iob[i][0] in ['heavy','big','such'] and iob[i-1][1] not in ['RB'] and iob[i+1][0] not in ['A','a']:
                seen[i] = iob[i][0]+" a"
                #eliminate next case 
                if i+1<len(iob) and iob[i+1][0] in ['The','the','An','an']: 
                    eliminate_next(i)
            
            elif iob[i-1][0] in ['The','the','A','a','An','an']:
                eliminate_prev(i,seen)


        #4. Article before adverb
        #4.1 for the format DT RB JJ -> a    and   if sentence starts with an adverb -> no article
        elif iob[i][1] == 'RB' and iob[i][0] not in ['as','not']:
            if i+1<len(iob) and iob[i+1][1] in ['JJ','RB'] and i>0:
                if i+2<len(iob) and iob[i+2][1] not in ['.'] and iob[i-1][0] not in ['A','a']:
                    seen[i] = "a "+iob[i][0]
                    if iob[i-1][0] in ['The','the','An','an']: 
                        eliminate_prev(i,seen)
                elif i+2<len(iob) and iob[i+2][1] in ['.'] and iob[i-1][0] in ['A','a']:
                    eliminate_prev(i,seen)
                
            elif i+2<len(iob) and iob[i+1][1] in ['VBG'] and iob[i+2][1] in ['NN'] and iob[i-1][0] not in ['A','a']:
                seen[i] = "a "+iob[i][0]
                if iob[i-1][0] in ['The','the','An','an']: 
                    eliminate_prev(i,seen)
            

        elif iob[i][1] == 'RBR' and iob[i-1][1] =='VBZ':
            if i+1<len(iob) and iob[i+1][1] in ['NN','NNP']:
                seen[i] = 'a '+iob[i][0]
                if iob[i-1][0] in ['The','the','An','an']: 
                    eliminate_prev(i,seen)
                    
                    
        #For Verbs
        elif i+1<len(iob) and iob[i][1] in ['VBG'] and iob[i+1][1] in ['PRP'] and iob[i][0] not in ['The','the']:
            seen[i]="the "+iob[i][0]
            if iob[i-1][0] in ['A','a','An','an']: 
                eliminate_prev(i,seen)
        
        elif i+1<len(iob) and iob[i][1] in ['VBG'] and iob[i+1][1] in ['NN']:
            if iob[i-1][0] not in ['a','A'] and iob[i][0] in countable and iob[i-1][1] not in ['IN']:
                seen[i]='a '+iob[i][0]
                if iob[i-1][0] in ['The','the','An','an']: 
                    eliminate_prev(i,seen)
                
        elif i+1<len(iob) and iob[i][1] in ['VBN'] and iob[i-1][1] in ['VBP'] and (iob[i-2][1] not in ['PRP'] or iob[i][0] in countable) and iob[i+1][1] in ['.'] and iob[i-1][0] not in ['a','A']:
            seen[i] = 'a '+iob[i][0]
            if iob[i-1][0] in ['The','the','An','an']: 
                eliminate_prev(i,seen)
                
        #print(seen)
    
    for i in range(len(seen)):
        if i+1<len(seen) and (seen[i+1] not in ['.',',','?','!',';',"'",':','(',')','[',']','/'] and seen[i+1].startswith("'")==False) and seen[i]!='' and i<len(iob) and "'" not in iob[i+1][0]:
            output+=seen[i]+' '
        else:
            output+=seen[i]
        
    return output

In [363]:
def eliminate_prev(i,seen):
    for x in range(i):
        if x == i-1:
            seen[x]=''
                    
def eliminate_next(i,seen):        
    for x in range(i):
        if x== i+1:
            seen[x]=''

In [353]:
#For Correction of sentence only.
inp = input()
output = Article_Error_Correction(inp)
print(output)

He is good boy.
He is a good boy.


In [364]:
test_df = pd.read_csv('test.csv')
test_input = test_df['Incorrect']
test_correct = test_df['Correct']
score = list()
for item,check in zip(test_input,test_correct):
    output = Article_Error_Correction(item)
    if output.lower()==check.lower():
        score.append(1)
    else:
        score.append(0)
print(score)
test_df["score"] = score
accuracy = (sum(score)/len(test_input))*100
print("Total Score: "+str(sum(score))+" out of "+str(len(test_input)))
print("Accuracy = ",accuracy)
test_df.to_csv('test_updated.csv',index=False)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Total Score: 60 out of 64
Accuracy =  93.75


In [365]:
#For Correction of paragraphs.
para = '''Ram is very nice guy. He always helps everyone. He has the enthusiastic personality. He wants to pursue his higher studies in United States of America'''

sentence_tokens = nltk.sent_tokenize(para)
print("Original Paragarph: ")
for t in sentence_tokens:
    print (t, "\n")
    
print("Output Paragraph: ")    
for t in sentence_tokens:   
    output=Article_Error_Correction(t)
    print(output,'\n')

Original Paragarph: 
Ram is very nice guy. 

He always helps everyone. 

He has the enthusiastic personality. 

He wants to pursue his higher studies in United States of America 

Output Paragraph: 
Ram is a very nice guy. 

He always helps everyone. 

He has an enthusiastic personality. 

He wants to pursue his higher studies in the United States of America 

